## JG Thoughts 
- Unemployment rate vs employment #s? Look at change in UE rate or total as of certain point in time
- Inc_Per_CAp_2022 -> typo?
- Population Density per mi² -> what yr? Also is string
- NA values in NJ
- Convert weeks into %?

- Maybe PCA for the white, black, Hispanic stats? But missing a decent chunk it looks like. Maybe only black or white. But a lot of 0s so maybe just exclde entirely  
- Maybe do something to 'Population' like catgy 1-5 for large small or medium. But can leave for now I guess 

- Do SVM or NN models? SVM mostly for classification. NN maybe not enough data

## Data Cleaning & Preprocessing

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import VotingRegressor

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor


In [2]:
df = pd.read_csv('Data/Cleaned/county_df2.csv')
df.head()

,FIPS_x,County,Years of Potential Life Lost Rate (premature death),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair/Poor Health,percent_smokers,percent_obese,Food Environment Index,...,cases_2020,cases_2021,cases_2022,deaths_2020,deaths_2021,deaths_2022,Masks,FIPS_y,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_65PlusPop_Pct
0,1001,Autauga,8824.0,10471.0,NaN,8707.0,18,19,38,7.2,...,4190.0,11018.0,18961.0,48.0,160.0,230.0,267.0,1001,42.2,73.8
1,1003,Baldwin,7225.0,10042.0,3087.0,7278.0,18,17,31,8.0,...,13601.0,39911.0,67496.0,161.0,593.0,719.0,267.0,1003,53.2,89.9
2,1005,Barbour,9586.0,11333.0,NaN,7310.0,26,22,44,5.6,...,1514.0,3860.0,7027.0,32.0,81.0,103.0,267.0,1005,44.5,75.3
3,1007,Bibb,11784.0,14813.0,NaN,11328.0,20,20,38,7.6,...,1834.0,4533.0,7692.0,46.0,95.0,108.0,267.0,1007,36.6,64.2
4,1009,Blount,10908.0,NaN,5620.0,11336.0,21,20,34,8.5,...,4641.0,11256.0,17731.0,63.0,198.0,260.0,267.0,1009,31.9,56.6


In [183]:
# df[['cases_2020', 'cases_2021', 'cases_2022', 'deaths_2020', 'deaths_2021', 'deaths_2022', 'Population']]

In [3]:
# Drop rows that we will not be using 'Segregation index black/white',
df.drop(columns = ['County', 'YPLL Rate (Black)', 'YPLL Rate (Hispanic)', 'YPLL Rate (White)', 'Number Uninsured', 'Number Primary Care Physicians', 'FIPS_y', 'Number pre-mature Deaths',
                        'Preventable Hosp. Rate (Black)', 'Preventable Hosp. Rate (Hispanic)', 'Preventable Hosp. Rate (White)',  'Percent Vaccinated Flu (Black)', 'Percent Uninsured',
                        'Percent  Vaccinated (Hispanic) Flu', 'Percent Vaccinated (White) Flu', 'Number Some College', 'Number Unemployed', 'Labor Force', 'PCP Ratio', 
                        '80th Percentile Income', '20th Percentile Income', '95% CI - Low', '95% CI - High', 'Life Expectancy (Black)', 'Life Expectancy (Hispanic)', 
                        'Life Expectancy (White)', 'Number HIV Cases', 'Household income (Black)', 'Household income (Hispanic)', 'Household income (White)'], inplace = True)

In [4]:
# df.columns
# df.dtypes
# df.shape

In [5]:
# Make FIPS index 
df.set_index('FIPS_x', inplace=True)

In [89]:
# # Create new columns for per populaltion stats - YPL, Number pre-mature Deaths, Number rural 
# df['YPL'] = df['Years of Potential Life Lost Rate (premature death)']/df['Population']
# df['pre mature deaths'] = df['Number pre-mature Deaths']/df['Population']
# df['rural'] = df['number Rural']/df['Population']

# df.drop(columns = ['Years of Potential Life Lost Rate (premature death)', 'Number pre-mature Deaths', 'number Rural'], inplace = True)

In [6]:
# Dummify State and Presence of water violation
# df = pd.get_dummies(columns = ['State'], data = df, drop_first=True)
df['water'] = df['Presence of water violation'].map({'No': 0, 'Yes': 1})
df.drop(columns = ['Presence of water violation', 'State'], inplace = True)

In [7]:
# Calculate total cases and deaths, and convert to % of population 
df['cases'] = df['cases_2022']
df['deaths'] = df['deaths_2022']

# df['case_rate'] = df['cases']/df['Population']
# df['death_rate'] = df['deaths']/df['Population']

# Deaths seem to be off? More deaths than population 
df.drop(columns = ['cases_2020', 'cases_2021', 'cases_2022', 'deaths_2020', 'deaths_2021', 'deaths_2022'], inplace = True)

In [8]:
# Drop Na values (3k rows -> 1828)
df.dropna(inplace=True)
df.shape

(1828, 47)

## Linear Regression (LR)

In [76]:
# y = df['cases']
# # y = df['deaths']
# # X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
# #        'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# X = df.drop(columns = ['cases', 'deaths'])
# ols = sm.OLS(y, X).fit()
# ols.summary()

In [82]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables               'Administered_Dose1_Recip_65PlusPop_Pct'
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks', 'Administered_Dose1_Pop_Pct']]
# X = df.drop(columns = ['cases', 'deaths'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [83]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [84]:
print(f'Train Score: {lr.score(X_train, y_train)}')
print(f'Test Score: {lr.score(X_test, y_test)}')

Train Score: 0.9639786858046525
Test Score: 0.932502060671604


In [80]:
pd.DataFrame(set(zip(X.columns, lr.coef_)), columns = ['Variable Name', 'Coefficient']).sort_values('Coefficient').round(10)

,Variable Name,Coefficient
7,percent Asian,-41.159927
2,Average Daily PM2.5,-19.457074
3,Percent Uninsured Adults,-5.445841
4,Percent Unemployed,-3.640236
0,Administered_Dose1_Pop_Pct,-0.158007
5,Masks,-0.015517
6,Population,0.003168
9,% Physically Inactive,13.657660
8,Percent Insufficient Sleep,17.013459
1,percent Not Proficient in English,31.305783


## K Nearest Neighbors (KNN)

In [85]:
# y variable will be cases or deaths
# y = df['cases']
y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [86]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [87]:
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train_sc, y_train)

KNeighborsRegressor(n_neighbors=10)

In [88]:
print(f'Train Score: {knn.score(X_train_sc, y_train)}')
print(f'Test Score: {knn.score(X_test_sc, y_test)}')

Train Score: 0.6639593331164743
Test Score: 0.7119991069508946


## Decision Trees (DT)

In [92]:
# y variable will be cases or deaths
# y = df['cases']
y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [95]:
%%time
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 13.2 ms


DecisionTreeRegressor()

In [96]:
print(f'Train Score: {tree.score(X_train, y_train)}')
print(f'Test Score: {tree.score(X_test, y_test)}')

Train Score: 1.0
Test Score: 0.8918904809334338


In [97]:
# Top Features
pd.DataFrame({'feature': X.columns, 'importance': tree.feature_importances_}).sort_values('importance', ascending = False)

,feature,importance
5,Population,0.930262
4,Percent Uninsured Adults,0.025897
6,percent Asian,0.018682
0,% Physically Inactive,0.011907
2,Average Daily PM2.5,0.003440
7,percent Not Proficient in English,0.003057
3,Percent Insufficient Sleep,0.003012
1,Percent Unemployed,0.002352
8,Masks,0.001391


### Bagged Decision Tree

In [103]:
# y variable will be cases or deaths
# y = df['cases']
y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [104]:
pgrid = {
    'estimator__max_depth': np.append(np.arange(1, 100), None),
}

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
bag = BaggingRegressor(tree,random_state = 42)
tree_bagged = GridSearchCV(bag, pgrid, cv = 5, n_jobs = 8)
# tree_bagged.get_params()

In [105]:
%%time
tree_bagged.fit(X_train, y_train)

CPU times: total: 203 ms
Wall time: 7.51 s


GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                        random_state=42),
             n_jobs=8,
             param_grid={'estimator__max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object)})

In [106]:
print(f'Train Score: {tree_bagged.score(X_train, y_train)}')
print(f'Test Score: {tree_bagged.score(X_test, y_test)}')

Train Score: 0.8827291185764962
Test Score: 0.792280327559715


In [107]:
tree_bagged.best_params_

{'estimator__max_depth': 17}

## Random Forest (RF)

In [136]:
# y variable will be cases or deaths
# y = df['cases']
y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [137]:
rf = RandomForestRegressor(
    random_state = 42,
)

In [138]:
%%time
rf.fit(X_train, y_train)

CPU times: total: 891 ms
Wall time: 891 ms


RandomForestRegressor(random_state=42)

In [139]:
print(f'Train Score: {rf.score(X_train, y_train)}')
print(f'Test Score: {rf.score(X_test, y_test)}')

Train Score: 0.9468652332831198
Test Score: 0.8121540003670638


In [140]:
pd.DataFrame({'Features': X.columns, 'Importance': rf.feature_importances_}).sort_values('Importance', ascending = False)

,Features,Importance
5,Population,0.923528
0,% Physically Inactive,0.016651
6,percent Asian,0.014197
4,Percent Uninsured Adults,0.013008
3,Percent Insufficient Sleep,0.008314
2,Average Daily PM2.5,0.008033
7,percent Not Proficient in English,0.006464
1,Percent Unemployed,0.006415
8,Masks,0.003390


### RF w/ RandomSearchCV

In [204]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [205]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': np.append(np.arange(1, 100), None),
}

# params = {
#  'bootstrap': [True, False],
#  'max_depth': np.append(np.arange(1, 100), None),
#  'max_features': ['log2', 'sqrt'],
#  'min_samples_leaf': np.arange(1, 10),
#  'min_samples_split': np.arange(1, 20),
#  'n_estimators': [50, 100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
# }

rf = RandomForestRegressor(
    random_state = 42,
)


rf_rs = RandomizedSearchCV(rf, params, n_iter=1000, cv = 5, n_jobs = 8)
# rf_rs.get_params()

In [206]:
%%time
rf_rs.fit(X_train, y_train)

CPU times: total: 4.44 s
Wall time: 6min 27s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=1000, n_jobs=8,
                   param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                        'max_features': array([5, 6, 7, 8, 9]),
                                        'min_samples_leaf': [1, 2, 3, 4, 5]})

In [207]:
print(f'Train Score: {rf_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_rs.score(X_test, y_test)}')

Train Score: 0.9475963064487434
Test Score: 0.8313269452770706


In [208]:
rf_rs.best_params_

{'min_samples_leaf': 1, 'max_features': 9, 'max_depth': 18}

### RF w/ Gradboost

In [151]:
# y variable will be cases or deaths
# y = df['cases']
y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [152]:
rf_gb = GradientBoostingRegressor()

In [153]:
%%time
rf_gb.fit(X_train, y_train)

CPU times: total: 172 ms
Wall time: 197 ms


GradientBoostingRegressor()

In [154]:
print(f'Train Score: {rf_gb.score(X_train, y_train)}')
print(f'Test Score: {rf_gb.score(X_test, y_test)}')

Train Score: 0.9938920088512411
Test Score: 0.9222647584908638


#### RF w/ Gradboost & RandomSearchCV

In [209]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [210]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': np.append(np.arange(1, 100), None),
}

rf_gb = GradientBoostingRegressor()


rf_gb_rs = RandomizedSearchCV(rf_gb, params, n_iter=1000, cv = 5, n_jobs = 8)
# rf_gb_rs.get_params()

In [211]:
%%time
rf_gb_rs.fit(X_train, y_train)

CPU times: total: 2.2 s
Wall time: 12min 4s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=1000,
                   n_jobs=8,
                   param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                        'max_features': array([5, 6, 7, 8, 9]),
                                        'min_samples_leaf': [1, 2, 3, 4, 5]})

In [212]:
print(f'Train Score: {rf_gb_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_gb_rs.score(X_test, y_test)}')

Train Score: 0.9946659198096452
Test Score: 0.8852267061850893


In [213]:
rf_gb_rs.best_params_

{'min_samples_leaf': 1, 'max_features': 9, 'max_depth': 2}

## Extra Trees (ET)

In [168]:
# y variable will be cases or deaths
# y = df['cases']
y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [169]:
%%time
et = ExtraTreesRegressor()
et.fit(X_train, y_train)

CPU times: total: 484 ms
Wall time: 496 ms


ExtraTreesRegressor()

In [170]:
print(f'Train Score: {et.score(X_train, y_train)}')
print(f'Test Score: {et.score(X_test, y_test)}')

Train Score: 0.9999999999987781
Test Score: 0.8851501149784387


#### ET w/ RandomSearchCV

In [214]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [215]:
params = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': np.append(np.arange(1, 100), None),
}


et = ExtraTreesRegressor()

et_rs = RandomizedSearchCV(et, params, n_iter=1000, cv = 5, n_jobs = 8)
# et_rs.get_params()

In [216]:
%%time
et_rs.fit(X_train, y_train)

CPU times: total: 2.95 s
Wall time: 3min 2s


RandomizedSearchCV(cv=5, estimator=ExtraTreesRegressor(), n_iter=1000, n_jobs=8,
                   param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                        'max_features': array([5, 6, 7, 8, 9]),
                                        'min_samples_leaf': [1, 2, 3, 4, 5]})

In [217]:
print(f'Train Score: {et_rs.score(X_train, y_train)}')
print(f'Test Score: {et_rs.score(X_test, y_test)}')

Train Score: 1.0
Test Score: 0.8759890389104943


In [218]:
et_rs.best_params_

{'min_samples_leaf': 1, 'max_features': 6, 'max_depth': 49}

#### Extra Trees w/ Grid Search 

In [17]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
X = df.drop(columns = ['cases', 'deaths'])
# X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       # 'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [18]:
pgrid = {
    'max_features': np.arange(5, X.shape[1] + 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': np.append(np.arange(1, 100), None),
}

et = ExtraTreesRegressor()
et.fit(X_train, y_train)
et_gs = GridSearchCV(et, pgrid, cv = 5, n_jobs = 8)
# et_gs.get_params()

In [19]:
%%time
et_gs.fit(X_train, y_train)

CPU times: total: 1h 20min 10s
Wall time: 2h 48min 49s


GridSearchCV(cv=5, estimator=ExtraTreesRegressor(), n_jobs=8,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                         'max_features': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45]),
                         'min_samples_leaf': [1, 2, 3, 4, 5]})

In [20]:
print(f'Train Score: {et_gs.score(X_train, y_train)}')
print(f'Test Score: {et_gs.score(X_test, y_test)}')

Train Score: 0.9998073803560398
Test Score: 0.8392267315542459


In [21]:
et_gs.best_params_

{'max_depth': 13, 'max_features': 35, 'min_samples_leaf': 1}

## Ensembled Model (Voting Regressor)

In [219]:
# y variable will be cases or deaths
y = df['cases']
# y = df['deaths']

# X variables
# X = df.drop(columns = ['cases', 'deaths'])
X = df[['% Physically Inactive', 'Percent Unemployed', 'Average Daily PM2.5', 'Percent Insufficient Sleep', 'Percent Uninsured Adults',
       'Population', 'percent Asian', 'percent Not Proficient in English', 'Masks']]
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [220]:
vr = VotingRegressor([
    ('lr', lr),
    ('tree', tree_bagged),
    ('rf_rs', rf_rs),
    ('rf_gb', rf_gb),
    ('et_rs', et_rs)
])

In [221]:
%%time
vr.fit(X_train, y_train)

CPU times: total: 5.75 s
Wall time: 9min 47s


VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('tree',
                             GridSearchCV(cv=5,
                                          estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                                                     random_state=42),
                                          n_jobs=8,
                                          param_grid={'estimator__max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,...
                                                param_distributions={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, None], dtype=object),
                                                                     'max_features': array([5, 6, 7, 8, 9]),
                                                                     'min_samples_leaf': [1,
                                                                                          2,
                                                                                          3,
                                                                                          4,
                                                                                          5]}))])

In [222]:
print(f'Train Score: {vr.score(X_train, y_train)}')
print(f'Train Score: {vr.score(X_test, y_test)}')

Train Score: 0.9805737869170732
Train Score: 0.883554965650196
